In [31]:
from dotenv import load_dotenv
load_dotenv()


import os 
import pinecone
print(pinecone.__version__)
import openai
from pinecone import Pinecone, ServerlessSpec

import sys
print(sys.executable)



5.0.1
/Users/rabiaghafoor/miniconda3/envs/rag/bin/python


In [39]:
load_dotenv('.env.local')
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")

)